In [1]:
import torch

In [2]:
import src.config as config

cfg = config.load_config("./configs/Replica/room0.yaml", "./configs/nice_slam.yaml")

In [3]:
import src.conv_onet.models.decoder as decoder
from torchsummary import summary

nn = decoder.NICE(hidden_size=16, coarse=True).to("cuda:0")


In [6]:
summary(decoder.NICE(hidden_size=16, coarse=True))

Layer (type:depth-idx)                             Param #
├─MLP_no_xyz: 1-1                                  --
|    └─ModuleList: 2-1                             --
|    |    └─DenseLayer: 3-1                        528
|    |    └─DenseLayer: 3-2                        272
|    |    └─DenseLayer: 3-3                        272
|    |    └─DenseLayer: 3-4                        784
|    |    └─DenseLayer: 3-5                        272
|    └─DenseLayer: 2-2                             17
├─MLP: 1-2                                         --
|    └─ModuleList: 2-3                             --
|    |    └─Linear: 3-6                            528
|    |    └─Linear: 3-7                            528
|    |    └─Linear: 3-8                            528
|    |    └─Linear: 3-9                            528
|    |    └─Linear: 3-10                           528
|    └─GaussianFourierFeatureTransform: 2-4        279
|    └─ModuleList: 2-5                             --
|    |    └─

Layer (type:depth-idx)                             Param #
├─MLP_no_xyz: 1-1                                  --
|    └─ModuleList: 2-1                             --
|    |    └─DenseLayer: 3-1                        528
|    |    └─DenseLayer: 3-2                        272
|    |    └─DenseLayer: 3-3                        272
|    |    └─DenseLayer: 3-4                        784
|    |    └─DenseLayer: 3-5                        272
|    └─DenseLayer: 2-2                             17
├─MLP: 1-2                                         --
|    └─ModuleList: 2-3                             --
|    |    └─Linear: 3-6                            528
|    |    └─Linear: 3-7                            528
|    |    └─Linear: 3-8                            528
|    |    └─Linear: 3-9                            528
|    |    └─Linear: 3-10                           528
|    └─GaussianFourierFeatureTransform: 2-4        279
|    └─ModuleList: 2-5                             --
|    |    └─

In [5]:
import numpy as np

def load_pretrain(nice: decoder.NICE, cfg: dict):
    ckpt = torch.load(cfg['pretrained_decoders']['coarse'], map_location=cfg['mapping']['device'])
    coarse_dict = {}
    for key, val in ckpt['model'].items():
        if ('decoder' in key) and ('encoder' not in key):
            key = key[8:]
            coarse_dict[key] = val
    nice.coarse_decoder.load_state_dict(coarse_dict)

    ckpt = torch.load(cfg['pretrained_decoders']['middle_fine'], map_location=cfg['mapping']['device'])
    middle_dict = {}
    fine_dict = {}
    for key, val in ckpt['model'].items():
        if ('decoder' in key) and ('encoder' not in key):
            if 'coarse' in key:
                key = key[8+7:]
                middle_dict[key] = val
            elif 'fine' in key:
                key = key[8+5:]
                fine_dict[key] = val
    nice.middle_decoder.load_state_dict(middle_dict)
    nice.fine_decoder.load_state_dict(fine_dict)

    scale = cfg['scale']

    bound = torch.from_numpy(np.array(cfg['mapping']['bound'])*scale)
    bound_divisible = cfg['grid_len']['bound_divisible']
    bound[:, 1] = (((bound[:, 1]-bound[:, 0]) /
        bound_divisible).int()+1)*bound_divisible+bound[:, 0]

    nice.bound = bound
    nice.middle_decoder.bound = bound
    nice.fine_decoder.bound = bound
    nice.color_decoder.bound = bound
    nice.coarse_decoder.bound = bound * cfg['model']['coarse_bound_enlarge']

In [6]:
load_pretrain(nn, cfg)

In [17]:
import os

stages = ["coarse", "middle", "fine", "color"]

for stage in stages[:1]:
    for pt in os.listdir(f"./saved_inputs/{stage}")[:1]:
        data = torch.load(f"./saved_inputs/{stage}/{pt}")
        pi = data['pi'].to(cfg['mapping']['device'])
        c = data['c']
        # print(summary(nn, input_data=(pi, c[f"grid_{stage}"], stage)))
        print(pi.shape, c[f"grid_{stage}"].shape)

torch.Size([1, 32000, 3]) torch.Size([1, 32, 7, 8, 11])
